# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [94]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import anthropic
from anthropic.types import ToolUseBlock

In [129]:
load_dotenv(override=True)

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY','***')
openai_api_key = os.getenv('OPENAI_API_KEY','****')

if anthropic_api_key:
    print(f"Anthropic API key exists and starts with {anthropic_api_key[:7]}")
else:
    print("Please provide anthropic api key")


if openai_api_key:
    print(f"OpenAi API key exists and starts with {openai_api_key[:8]}")
else:
    print("Please provide openai api key")

Anthropic API key exists and starts with sk-ant-
OpenAi API key exists and starts with sk-proj-


In [6]:
model_name = 'claude-3-haiku-20240307'

claude = anthropic.Anthropic(api_key = anthropic_api_key)

In [42]:
messages = []

## Tools


In [7]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [8]:
get_ticket_price("london")

Tool get_ticket_price called for london


'$799'

In [114]:
pip install gtts playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7099 sha256=106a7a52e925fd11af01e95c8956ede7fea938f21aacc0039116ae37d9d42bd1
  Stored in directory: /Users/tanayakarmakar/Library/Caches/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.


In [133]:
from pydub import AudioSegment
from pydub.playback import play
from openai import OpenAI
from io import BytesIO


openai = OpenAI(api_key = openai_api_key)

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)
    

In [135]:
talker("tell me a cat joke")

Input #0, wav, from '/var/folders/f4/v6qdzld96kgdrq05w5ghmrp40000gn/T/tmpf4ok7m7y.wav':
  Duration: 00:00:01.18, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.06 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

In [29]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "input_schema": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [31]:
tools = [price_function]

In [11]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [105]:
def handle_tool_calls(tool_calls):
    print(tool_calls)
    tool_call = tool_calls.name
    print(f"Tool name {tool_call}")

    arguments =tool_calls.input
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "assistant",
        "content": json.dumps({"destination_city": city,"price": price})
    }
    return response, city 

In [108]:
def chat(message, history):
    global messages

    messages = []
    for user, assistant in history:
        messages.append({"role":"user", "content": user})
        messages.append({"role":"assistant","content": assistant})
    messages.append({"role": "user","content": message})

    print(messages)
    
    response = claude.messages.create(
        model= model_name,
        max_tokens=1000,
        system = system_message,
        messages = messages,
        tools = tools
    )

    tool_usage = None
    print(response)
    print(response.content[0].type)

    if response.content[0].type == 'tool_use':
        response, city = handle_tool_calls(response.content[0])
        messages.append(response)
        #messages.append(city)
        response = claude.messages.create(
            model = model_name,
            max_tokens = 100,
            messages = messages
        )
    messages.append({"role": "assistant","content":response.content[0].text })
    return response.content[0].text
        

In [110]:
chatbot = gr.ChatInterface(
    fn=chat,
    title="Claude Chatbot 🤖",
    chatbot=gr.Chatbot(),
    textbox=gr.Textbox(placeholder="Ask Claude anything..."),
)

chatbot.launch()


/var/folders/f4/v6qdzld96kgdrq05w5ghmrp40000gn/T/ipykernel_13842/2994664130.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(),
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


## Agent Framework

In [241]:
def translate(text, target_language):
    prompt = f"Translate the following sentence into {target_language}:\n\n\"{text}\""
    response = claude.messages.create(
        model=model_name,
        max_tokens=256,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt}
                ]
            }
        ]
    )

    print(response.content[0])

    return response.content[0].text
    

In [243]:
translate("hi there", "de")

TextBlock(citations=None, text='Hallo da', type='text')


'Hallo da'

In [249]:
def chat_with_claude(message, target_language,history):
    global messages

    print(f"Target language is {target_language}")

    messages = []
    if history:
        for user, assistant in history:
            messages.append({"role":"user", "content": user})
            messages.append({"role":"assistant","content": assistant})
    messages.append({"role": "user","content": message})

    print(messages)
    
    response = claude.messages.create(
        model= model_name,
        max_tokens=1000,
        system = system_message,
        messages = messages,
        tools = tools
    )

    tool_usage = None
    print(response)
    print(response.content[0].type)

    reply = ''
    if response.content[0].type == 'tool_use':
        response, city = handle_tool_calls(response.content[0])
        messages.append(response)
        reply = f"The flight price is ${response['content']}"

        print(f"Tool response ${reply}")
        response = claude.messages.create(
            model = model_name,
            max_tokens = 100,
            messages = messages
        )
        print('Tools Response')
        print(response)
    else:
        reply = response.content[0].text
    messages.append({"role": "assistant","content":reply })
    talker(translate(reply, target_language))
    return reply

In [250]:
# More involved Gradio code as we're not using the preset Chat interface!
# Passing in inbrowser=True in the last line will cause a Gradio window to pop up immediately.
LANGUAGES = [
    "French", "Spanish", "German", "Italian", "Portuguese",
    "Hindi", "Japanese", "Korean", "Chinese (Simplified)",
    "Arabic", "Russian", "Turkish"
]


with gr.Blocks() as demo:
    gr.Markdown("### 🧠 Claude + Tools + TTS")
    with gr.Row():
        language_dropdown = gr.Dropdown(label="Target Language", choices=LANGUAGES, value="French")
        user_input = gr.Textbox(label="Ask Claude something...")
    with gr.Row():
        submit_btn = gr.Button("Submit")
    with gr.Row():
        response_output = gr.Textbox(label="Claude's Response")
        #audio_output = gr.Audio(label="Audio", type="filepath")

    submit_btn.click(chat_with_claude, inputs=[user_input, language_dropdown], outputs=[response_output])

demo.launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py:1024: UserWarning: Expected 3 arguments for function <function chat_with_claude at 0x1554b2e80>, received 2.
  warnings.warn(
/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py:1028: UserWarning: Expected at least 3 arguments for function <function chat_with_claude at 0x1554b2e80>, received 2.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7938

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Target language is Hindi
[{'role': 'user', 'content': 'hi'}]
Message(id='msg_012XfkRCP12NEBzVtN96wLmS', content=[TextBlock(citations=None, text='Hello, how can I assist you today?', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=433, output_tokens=12))
text
TextBlock(citations=None, text='Here is the translation of the sentence "Hello, how can I assist you today?" into Hindi:\n\n"नमस्ते, आज मैं आपकी किस प्रकार से मदद कर सकता हूं?"', type='text')


Input #0, wav, from '/var/folders/f4/v6qdzld96kgdrq05w5ghmrp40000gn/T/tmpyq9wmzcq.wav':
  Duration: 00:00:09.14, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   9.09 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


Target language is Hindi
[{'role': 'user', 'content': 'flight ticket to london'}]
Message(id='msg_01KRLgNwpcuhxQhKJnSX54aP', content=[ToolUseBlock(id='toolu_012JCREz1NXtmQbwyHiHhGPL', input={'destination_city': 'London'}, name='get_ticket_price', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=436, output_tokens=57))
tool_use
ToolUseBlock(id='toolu_012JCREz1NXtmQbwyHiHhGPL', input={'destination_city': 'London'}, name='get_ticket_price', type='tool_use')
Tool name get_ticket_price
Tool get_ticket_price called for London
Tool response $The flight price is ${"destination_city": "London", "price": "$799"}
Tools Response
Message(id='msg_014XZvGHbHYKdvDh3geBZePo', content=[], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, ca

Input #0, wav, from '/var/folders/f4/v6qdzld96kgdrq05w5ghmrp40000gn/T/tmpnomew5n_.wav':
  Duration: 00:00:04.56, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   4.46 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 